In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 컴퓨터 그래픽스, 기독교, 야구
cats = ['comp.graphics', 'soc.religion.christian','rec.sport.baseball']

news_df = fetch_20newsgroups(
    subset='all',
    remove=('headers', 'footers', 'quotes'),
    categories=cats,
    random_state=42
)

# 코퍼스
news_df

In [25]:
# 텍스트 전처리 (lemmatize, stopwords, 특수기호 제거 등) 필요하다 => 추가로 따로 해주기

topic modeling 은 DTM 사용
* 확률을 구해야 하기 때문에 빈도수가 중요
* TF-IDF를 사용하면 제대로 된 토픽 모델링이 안될 가능성이 커진다
* **확률을 구할때는 빈도수**

In [26]:
count_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2,
    max_features=1000, # 단어를 최대 몇 개 사용할 것인지
    stop_words='english',
    ngram_range=(1,2)
)

dtm_vector = count_vectorizer.fit_transform(news_df['data'])
dtm_vector.shape

(2964, 1000)

In [ ]:
count_vectorizer.get_feature_names_out()

## LDA 적용

In [28]:
lda = LatentDirichletAllocation(
    n_components=len(cats), # 사용할 토픽의 갯수 지정
    random_state=42
)

lda.fit(dtm_vector)

LatentDirichletAllocation(n_components=3, random_state=42)

## 주제별 단어 연관도 확인
* 토픽에 포함되는 단어의 연관도
  

In [29]:
lda.components_
# 각 토픽에 대해 1000개 단어의 연관도

array([[ 63.63332482,   0.34963495,  17.44704079, ...,  16.61505824,
          2.5384632 , 122.4210569 ],
       [208.02174752,  68.31556307,  90.89958785, ...,  80.29277085,
         54.27731685,   3.2448167 ],
       [  0.34492766,   0.33480198,   0.65337136, ...,   2.09217091,
         49.18421995,   0.3341264 ]])

In [30]:
# 각 토픽 별 중심 단어(점수가 높은 단어) 확인
for topic_index, topic_score in enumerate(lda.components_): # enumerate : 인덱스와 해당 요소를 동시에 얻을 수 있음
    print("Topic : {}".format(topic_index))
    # 단어의 인덱스 구하기
    topic_word_index = topic_score.argsort()[::-1] # argsort : 배열을 정렬했을 때 원본 배열의 인덱스를 반환
    top_index = topic_word_index[:10]

    words = count_vectorizer.get_feature_names_out()
    top_words = [ words[i] for i in top_index ]

    print(top_words)
    print()

Topic : 0
['image', 'graphics', 'jpeg', 'edu', 'file', 'images', 'data', 'available', 'software', 'use']

Topic : 1
['year', 'game', 'don', 'think', 'good', 'team', 'time', 'games', 'just', 'like']

Topic : 2
['god', 'people', 'jesus', 'church', 'think', 'know', 'does', 'just', 'christ', 'don']



## 각 문서 별 토픽 분포 확인

In [38]:
doc_topic = lda.transform(dtm_vector)
doc_topic

array([[0.85839148, 0.12070316, 0.02090536],
       [0.90436513, 0.05643273, 0.03920214],
       [0.22961015, 0.5236614 , 0.24672845],
       ...,
       [0.00392839, 0.00412269, 0.99194892],
       [0.92202287, 0.03668682, 0.04129031],
       [0.00184472, 0.00167317, 0.99648212]])

In [32]:
def get_filename_list(newsdata):
    filename_list=[]

    for file in newsdata.filenames:
            #print(file)
            filename_temp = file.split('\\')[-2:]
            filename = '.'.join(filename_temp)
            filename_list.append('/'.join(filename.split('/')[-2:]))

    return filename_list

filename_list = get_filename_list(news_df)
print("filename 개수:",len(filename_list), "filename list 10개만:",filename_list[:10])

filename 개수: 2964 filename list 10개만: ['rec.sport.baseball.105154', 'comp.graphics.38805', 'rec.sport.baseball.104616', 'comp.graphics.37928', 'rec.sport.baseball.104823', 'comp.graphics.38908', 'comp.graphics.38659', 'comp.graphics.38691', 'comp.graphics.38876', 'comp.graphics.38700']


In [33]:
import pandas as pd

topic_names = ["topic # {}".format(i) for i in range(0, len(cats))]

topic_df = pd.DataFrame(doc_topic, columns=topic_names, index=filename_list)
topic_df

,topic # 0,topic # 1,topic # 2
rec.sport.baseball.105154,0.858391,0.120703,0.020905
comp.graphics.38805,0.904365,0.056433,0.039202
rec.sport.baseball.104616,0.229610,0.523661,0.246728
comp.graphics.37928,0.333333,0.333333,0.333333
rec.sport.baseball.104823,0.028380,0.940660,0.030959
...,...,...,...
comp.graphics.38311,0.424100,0.416729,0.159172
comp.graphics.38435,0.333333,0.333333,0.333333
soc.religion.christian.20894,0.003928,0.004123,0.991949
comp.graphics.38275,0.922023,0.036687,0.041290


* 단어를 많이 쓰지도 않았고, 기사의 길이가 짧을 수 있어 정확한 값은 아니다

In [34]:
test = "I prayed to God to win this game. God answered, and we could win this game."
# 기독교와 야구에 관한 단어들 포함

In [35]:
test_vector = count_vectorizer.transform([test]) # 2차원 벡터로 삽입해야 한다
lda.transform(test_vector)

array([[0.04767431, 0.61900264, 0.33332305]])